### Chargement du dataset

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Chargement du dataset
df = pd.read_csv("data/heart_2022_with_nans.csv")

# Aperçu global
print("Shape:", df.shape)
display(df.head())

# Types de données
display(df.dtypes)

Shape: (445132, 40)


,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,...,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos
0,Alabama,Female,Very good,0.0,0.0,Within past year (anytime less than 12 months ...,No,8.0,NaN,No,...,NaN,NaN,NaN,No,No,Yes,No,"Yes, received tetanus shot but not sure what type",No,No
1,Alabama,Female,Excellent,0.0,0.0,NaN,No,6.0,NaN,No,...,1.60,68.04,26.57,No,No,No,No,"No, did not receive any tetanus shot in the pa...",No,No
2,Alabama,Female,Very good,2.0,3.0,Within past year (anytime less than 12 months ...,Yes,5.0,NaN,No,...,1.57,63.50,25.61,No,No,No,No,NaN,No,Yes
3,Alabama,Female,Excellent,0.0,0.0,Within past year (anytime less than 12 months ...,Yes,7.0,NaN,No,...,1.65,63.50,23.30,No,No,Yes,Yes,"No, did not receive any tetanus shot in the pa...",No,No
4,Alabama,Female,Fair,2.0,0.0,Within past year (anytime less than 12 months ...,Yes,9.0,NaN,No,...,1.57,53.98,21.77,Yes,No,No,Yes,"No, did not receive any tetanus shot in the pa...",No,No


State                         object
Sex                           object
GeneralHealth                 object
PhysicalHealthDays           float64
MentalHealthDays             float64
LastCheckupTime               object
PhysicalActivities            object
SleepHours                   float64
RemovedTeeth                  object
HadHeartAttack                object
HadAngina                     object
HadStroke                     object
HadAsthma                     object
HadSkinCancer                 object
HadCOPD                       object
HadDepressiveDisorder         object
HadKidneyDisease              object
HadArthritis                  object
HadDiabetes                   object
DeafOrHardOfHearing           object
BlindOrVisionDifficulty       object
DifficultyConcentrating       object
DifficultyWalking             object
DifficultyDressingBathing     object
DifficultyErrands             object
SmokerStatus                  object
ECigaretteUsage               object
C

### Étape 1 : Aperçu des valeurs uniques

In [6]:
print(" Aperçu des valeurs uniques par colonne :\n")

for col in df.columns:
    uniques = df[col].unique()
    cleaned = [str(val).strip() if pd.notna(val) else "vide" for val in uniques]
    cleaned = ["vide" if val in ["", " ", "[]", "nan", "None"] else val for val in cleaned]
    cleaned = sorted(set(cleaned))

    if len(cleaned) > 10:
        print(f"{col}: {', '.join(cleaned[:5])}... [{len(cleaned)} valeurs uniques]")
    else:
        print(f"{col}: {', '.join(cleaned)}")

 Aperçu des valeurs uniques par colonne :

State: Alabama, Alaska, Arizona, Arkansas, California... [54 valeurs uniques]
Sex: Female, Male
GeneralHealth: Excellent, Fair, Good, Poor, Very good, vide
PhysicalHealthDays: 0.0, 1.0, 10.0, 11.0, 12.0... [32 valeurs uniques]
MentalHealthDays: 0.0, 1.0, 10.0, 11.0, 12.0... [32 valeurs uniques]
LastCheckupTime: 5 or more years ago, Within past 2 years (1 year but less than 2 years ago), Within past 5 years (2 years but less than 5 years ago), Within past year (anytime less than 12 months ago), vide
PhysicalActivities: No, Yes, vide
SleepHours: 1.0, 10.0, 11.0, 12.0, 13.0... [25 valeurs uniques]
RemovedTeeth: 1 to 5, 6 or more, but not all, All, None of them, vide
HadHeartAttack: No, Yes, vide
HadAngina: No, Yes, vide
HadStroke: No, Yes, vide
HadAsthma: No, Yes, vide
HadSkinCancer: No, Yes, vide
HadCOPD: No, Yes, vide
HadDepressiveDisorder: No, Yes, vide
HadKidneyDisease: No, Yes, vide
HadArthritis: No, Yes, vide
HadDiabetes: No, No, pre-diabet

### Étape 2 : Nettoyage des valeurs manquantes

In [7]:
# 🧽 Étape 1 : remplacer toutes les valeurs "vides" par NaN
valeurs_vides = ["", " ", "[]", "vide", None, np.nan]
df_clean = df.replace(valeurs_vides, np.nan)

In [8]:
# Étape 2 : traitement conditionnel selon taux de valeurs manquantes
print("Aperçu du traitement des valeurs manquantes par colonne :\n")

seuil = 20  # Ici, 20% → au-delà, on considère que la colonne doit être corrigée

for col in df_clean.columns:
    total = len(df_clean)
    n_missing = df_clean[col].isna().sum()
    pct_missing = (n_missing / total) * 100
# Si la colonne contient des valeurs manquantes
    if n_missing > 0:
        print(f" {col}: {n_missing} valeurs manquantes ({pct_missing:.1f}%)")

        if pct_missing >= seuil:
            # Si le taux de valeurs manquantes est important (≥ seuil défini),
            # on remplace les valeurs manquantes par la valeur la plus fréquente (mode)
            mode_val = df_clean[col].mode(dropna=True)
            if not mode_val.empty:
                df_clean[col] = df_clean[col].fillna(mode_val[0])
                print(f"   ↳ Remplacé par la valeur la plus fréquente : '{mode_val[0]}'")
            else:
                #complètement vide (100 % manquante)
                print(f"   ↳  Aucun mode détecté, colonne non modifiée.")
        else:
            # Si peu de valeurs manquantes (< seuil), on ne touche pas à la colonne
            # Cela évite d'introduire du bruit inutile dans les données
            print(f"   ↳  Manque peu de valeurs, laissé tel quel.")

Aperçu du traitement des valeurs manquantes par colonne :

 GeneralHealth: 1198 valeurs manquantes (0.3%)
   ↳  Manque peu de valeurs, laissé tel quel.
 PhysicalHealthDays: 10927 valeurs manquantes (2.5%)
   ↳  Manque peu de valeurs, laissé tel quel.
 MentalHealthDays: 9067 valeurs manquantes (2.0%)
   ↳  Manque peu de valeurs, laissé tel quel.
 LastCheckupTime: 8308 valeurs manquantes (1.9%)
   ↳  Manque peu de valeurs, laissé tel quel.
 PhysicalActivities: 1093 valeurs manquantes (0.2%)
   ↳  Manque peu de valeurs, laissé tel quel.
 SleepHours: 5453 valeurs manquantes (1.2%)
   ↳  Manque peu de valeurs, laissé tel quel.
 RemovedTeeth: 11360 valeurs manquantes (2.6%)
   ↳  Manque peu de valeurs, laissé tel quel.
 HadHeartAttack: 3065 valeurs manquantes (0.7%)
   ↳  Manque peu de valeurs, laissé tel quel.
 HadAngina: 4405 valeurs manquantes (1.0%)
   ↳  Manque peu de valeurs, laissé tel quel.
 HadStroke: 1557 valeurs manquantes (0.3%)
   ↳  Manque peu de valeurs, laissé tel quel.
 HadA

### Étape 3 : Conversion des colonnes catégorielles vers colonnes numériques (_nettoye)

In [ ]:
# Colonnes de type object à convertir
colonnes_object = df_clean.select_dtypes(include='object').columns.tolist()

print("\n Création de colonnes codées (_nettoye) pour les colonnes catégorielles :\n")

for col in colonnes_object:
    if df_clean[col].nunique() <= 50:  # On limite aux colonnes avec un nombre raisonnable de modalités
        mapping = {val: i for i, val in enumerate(df_clean[col].dropna().unique())}
        df_clean[col + "_nettoye"] = df_clean[col].map(mapping)
        print(f" {col} → {col + '_nettoye'} | Mapping : {mapping}")
    else:
        print(f" {col} ignorée (trop de modalités)")


🔁 Création de colonnes codées (_nettoye) pour les colonnes catégorielles :

⏭️ State ignorée (trop de modalités)
🔧 Sex → Sex_nettoye | Mapping : {'Female': 0, 'Male': 1}
🔧 GeneralHealth → GeneralHealth_nettoye | Mapping : {'Very good': 0, 'Excellent': 1, 'Fair': 2, 'Poor': 3, 'Good': 4}
🔧 LastCheckupTime → LastCheckupTime_nettoye | Mapping : {'Within past year (anytime less than 12 months ago)': 0, 'Within past 2 years (1 year but less than 2 years ago)': 1, 'Within past 5 years (2 years but less than 5 years ago)': 2, '5 or more years ago': 3}
🔧 PhysicalActivities → PhysicalActivities_nettoye | Mapping : {'No': 0, 'Yes': 1}
🔧 RemovedTeeth → RemovedTeeth_nettoye | Mapping : {'None of them': 0, '1 to 5': 1, '6 or more, but not all': 2, 'All': 3}
🔧 HadHeartAttack → HadHeartAttack_nettoye | Mapping : {'No': 0, 'Yes': 1}
🔧 HadAngina → HadAngina_nettoye | Mapping : {'No': 0, 'Yes': 1}
🔧 HadStroke → HadStroke_nettoye | Mapping : {'No': 0, 'Yes': 1}
🔧 HadAsthma → HadAsthma_nettoye | Mapping 

### Sauvegarder df_clean dans un nouveau fichier CSV

In [10]:
df_clean.to_csv("data/heart_2022_clean.csv", index=False)

 verifier si il y'a des valeurs aberrente , valeurs dupliquer
 